In [2]:
import re
import time 
import logging
import requests
import re
import sys
from bs4 import BeautifulSoup
from queue import Queue
from urllib import parse, request
from urllib.parse import urlparse
import heapq
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#### Coursera Scraping

Starting URL: https://www.coursera.org/search?query=computer%20science&topic=Computer%20Science&language=English&sortBy=BEST_MATCH

These are courses filtered by "Computer Science" and "English" and "courses"

Courses Format:https://www.coursera.org

Course1: https://www.coursera.org/learn/cs-programming-java
Course2: https://www.coursera.org/learn/html-css-javascript-for-web-developers

Course Format: https://www.coursera.org/learn/[CLASS NAME]


In [89]:
# URL of the search results page

site = "https://www.coursera.org/search?query=computer%20science&language=English&productTypeDescription=Courses&topic=Computer%20Science&subtitleLanguage=English&sortBy=BEST_MATCH"

In [90]:
links = []

# Create a webdriver instance
driver = webdriver.Chrome()

# Navigate to the first page of search results
driver.get(site)

while True:
    try:
        # Wait for the page to fully load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "cds-CommonCard-titleLink")))
        
        # Extract links from the current page
        cards = driver.find_elements(By.CLASS_NAME, 'cds-CommonCard-titleLink')
        for card in cards:
            link = card.get_attribute("href")
            links.append(link)
        #print(links)

        # Click the "Go to next page" button
        next_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//button[@aria-label='Go to next page']")))
        next_button.click()
        
    except Exception as e:
        # If there is no next button, break the loop
        print(f"No more pages: {e}")
        break

# Close the webdriver
driver.quit()


No more pages: Message: 
Stacktrace:
0   chromedriver                        0x000000010585bae8 chromedriver + 5217000
1   chromedriver                        0x0000000105853723 chromedriver + 5183267
2   chromedriver                        0x00000001053c5527 chromedriver + 406823
3   chromedriver                        0x0000000105410ff2 chromedriver + 716786
4   chromedriver                        0x0000000105411181 chromedriver + 717185
5   chromedriver                        0x00000001054551d4 chromedriver + 995796
6   chromedriver                        0x000000010543372d chromedriver + 857901
7   chromedriver                        0x00000001054526bf chromedriver + 984767
8   chromedriver                        0x00000001054334a3 chromedriver + 857251
9   chromedriver                        0x0000000105403fe3 chromedriver + 663523
10  chromedriver                        0x000000010540492e chromedriver + 665902
11  chromedriver                        0x0000000105819a00 chromedrive

In [91]:
print(len(links))

1000


In [75]:
# test 
testlinks = ["https://www.coursera.org/learn/cs-programming-java", "https://www.coursera.org/learn/cs-algorithms-theory-machines"]

In [86]:
# test

for link in testlinks:
                # Create a webdriver instance
            driver = webdriver.Chrome()

            # Navigate to the first page of search results
            driver.get(link)
            title_element = driver.find_element(By.CSS_SELECTOR, 'meta[property="og:title"]')
            title = title_element.get_attribute("content")
            print("Title:", title)
            
            description_element = driver.find_element(By.CSS_SELECTOR, 'div.content-inner')
            paragraphs = description_element.find_elements(By.TAG_NAME, 'p')
            description = ' '.join([p.text for p in paragraphs])
            print("Description:", description)


Title: Computer Science: Programming with a Purpose
Description: The basis for education in the last millennium was “reading, writing, and arithmetic;” now it is reading, writing, and computing. Learning to program is an essential part of the education of every student, not just in the sciences and engineering, but in the arts, social sciences, and humanities, as well. Beyond direct applications, it is the first step in understanding the nature of computer science’s undeniable impact on the modern world.  This course covers the first half of our book Computer Science: An Interdisciplinary Approach (the second half is covered in our Coursera course Computer Science: Algorithms, Theory, and Machines). Our intent is to teach programming to those who need or want to learn it, in a scientific context.  We begin by introducing basic programming elements such as variables, conditionals, loops, arrays, and I/O. Next, we turn to functions, introducing key concepts such as recursion, modular pro

In [92]:
# Visit each link and extract the title and description

titles = []
descriptions = []

for link in links:
    # Create a webdriver instance
    driver = webdriver.Chrome()

    # Navigate to the first page of search results
    driver.get(link)

    try:
        title_element = driver.find_element(By.CSS_SELECTOR, 'meta[property="og:title"]')
        title = title_element.get_attribute("content")
    except NoSuchElementException:
        print(f"Title element not found for {link}")
        continue

    try:
        description_element = driver.find_element(By.CSS_SELECTOR, 'div.content-inner')
        paragraphs = description_element.find_elements(By.TAG_NAME, 'p')
        description = ' '.join([p.text for p in paragraphs])
    except NoSuchElementException:
        print(f"Description element not found for {link}")
        continue

    titles.append(title)
    descriptions.append(description)

    driver.quit()


WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=124.0.6367.158)
Stacktrace:
0   chromedriver                        0x00000001030baae8 chromedriver + 5217000
1   chromedriver                        0x00000001030b2723 chromedriver + 5183267
2   chromedriver                        0x0000000102c24527 chromedriver + 406823
3   chromedriver                        0x0000000102c0be32 chromedriver + 306738
4   chromedriver                        0x0000000102c0ba9c chromedriver + 305820
5   chromedriver                        0x0000000102c09b7d chromedriver + 297853
6   chromedriver                        0x0000000102c09e9a chromedriver + 298650
7   chromedriver                        0x0000000102c26ebf chromedriver + 417471
8   chromedriver                        0x0000000102cb2255 chromedriver + 987733
9   chromedriver                        0x0000000102

In [103]:
len(titles)

177

In [106]:
titles_copy = titles
descriptions_copy = descriptions

In [108]:
# Visit each link and extract the title and description from the 177th course

for link in links[177:]:
    # Create a webdriver instance
    driver = webdriver.Chrome()

    # Navigate to the first page of search results
    driver.get(link)

    try:
        title_element = driver.find_element(By.CSS_SELECTOR, 'meta[property="og:title"]')
        title = title_element.get_attribute("content")
    except NoSuchElementException:
        print(f"Title element not found for {link}")
        continue

    try:
        description_element = driver.find_element(By.CSS_SELECTOR, 'div.content-inner')
        paragraphs = description_element.find_elements(By.TAG_NAME, 'p')
        description = ' '.join([p.text for p in paragraphs])
    except NoSuchElementException:
        print(f"Description element not found for {link}")
        continue

    titles.append(title)
    descriptions.append(description)

    driver.quit()


WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=124.0.6367.158)
Stacktrace:
0   chromedriver                        0x000000010242cae8 chromedriver + 5217000
1   chromedriver                        0x0000000102424723 chromedriver + 5183267
2   chromedriver                        0x0000000101f96527 chromedriver + 406823
3   chromedriver                        0x0000000101f7de32 chromedriver + 306738
4   chromedriver                        0x0000000101f7da9c chromedriver + 305820
5   chromedriver                        0x0000000101f7c7ae chromedriver + 300974
6   chromedriver                        0x0000000101fa200e chromedriver + 454670
7   chromedriver                        0x0000000102023a41 chromedriver + 985665
8   chromedriver                        0x00000001020044a3 chromedriver + 857251
9   chromedriver                        0x0000000101

In [111]:
len(descriptions)

307

In [112]:
coursera_df = pd.DataFrame(
    {'Title': titles,
     'Description': descriptions,
     'Link': links[:307]})

In [113]:
coursera_df.head()

,Title,Description,Link
0,Computer Science: Programming with a Purpose,The basis for education in the last millennium...,https://www.coursera.org/learn/cs-programming-...
1,"Computer Science: Algorithms, Theory, and Mac...",This course introduces the broader discipline...,https://www.coursera.org/learn/cs-algorithms-t...
2,"HTML, CSS, and Javascript for Web Developers",Do you realize that the only functionality of ...,https://www.coursera.org/learn/html-css-javasc...
3,Learn to Program: The Fundamentals,"Behind every mouse click and touch-screen tap,...",https://www.coursera.org/learn/learn-to-program
4,Introduction to Software Engineering,Are you curious about the lucrative field that...,https://www.coursera.org/learn/introduction-to...


In [114]:
coursera_df.to_csv('coursera_df_first307.csv')